In [1]:
import tatqa_utils
import pandas as pd

In [2]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [3]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2

In [4]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [5]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [7]:
def gen_code(llm, question, table):    
    question = question['question']
    prompt = f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: {question} \n Table: {table}"
    res = llm.invoke(prompt)
    code = res.content.replace('```python','').replace('```','')
    return (prompt, code)

def exec_code(code, table):  
    try: 
        loc = locals()   
        if not "run()" in code:
            exec(code + f"\nr = run({table})\n", globals(), loc)
        else: 
            exec(code + "\nr = run()\n", globals(), loc)
        return loc['r']
    except Exception as e:
            s = '[Error]'+ str(e)
            print(s)
            return (s,'')
#table = "[['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]"
#question = 'What is the change in Fixed Price in 2019 from 2018?'
table = "[['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]"
question = {'question':'What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?'}
( prompt, code) = gen_code(llm, question, table)
exec_code(code, table)

(14.065510597302506, '%')

In [15]:
from tqdm.notebook import tqdm as log_progress

res = []
for i, item in log_progress(devdf.iterrows()):
    table = item['table']['table']
    #print (i)
    for q in item['questions']:        
        if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']: 
            ( prompt, code)  = gen_code(llm, q, table)
            r = exec_code(code, table)
            err =  None
            if r is None:
                (pred_value, pred_unit) = (None, None)
            else:                
                if len(r) == 2:
                    (pred_value, pred_unit) = r                    
                    #print(pred_value, type(pred_value))
                    if isinstance(pred_value, str) and  pred_value.startswith('[Error]'):
                        (pred_value, pred_unit) = (None, None)
                        err,_ = r                                                        
                        
                elif len(r) == 1:
                    pred_value = r                
                else:
                    (pred_value, pred_unit) = (None, None)        
                
            #res.append({"table":table, "q":q, "pred":pred, "code": code})
            res.append(({"answer_type":q["answer_type"], "answer": q["answer"], 'scale': q["scale"]}, pred_value, pred_unit, q, code, prompt, item['table'], err))


0it [00:00, ?it/s]

[Error]could not convert string to float: ''
[Error]invalid literal for int() with base 10: '$592'
[Error]'(' was never closed (<string>, line 5)
[Error]list index out of range
[Error]invalid literal for int() with base 10: 'Selling, general and administrative'
[Error]invalid literal for int() with base 10: 'Operating income'
[Error]invalid literal for int() with base 10: '1,050'
[Error]leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<string>, line 2)
[Error]'Specific allowance for credit losses' is not in list
[Error]invalid literal for int() with base 10: '$18111'
[Error]could not convert string to float: '(3491626)'
[Error]unsupported operand type(s) for -: 'str' and 'str'
[Error]invalid literal for int() with base 10: '8,455'
[Error]invalid literal for int() with base 10: '(6134)'
[Error]name 'Table' is not defined
[Error]unterminated string literal (detected at line 3) (<string>, line 3)
[Error]'(' was never closed (<string>, line 

In [16]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=True)
print( pred_em, pred_f1, scale_score)

special case
special case
special case
special case
special case
special case
special case
0.20512820512820512 0.20512820512820512 0.03276353276353276


In [11]:
c = 0
for i, item in devdf.iterrows():
    table = item['table']['table']
    #print (i)
    for q in item['questions']:        
        if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']: 
            c = c+1
c

702

In [19]:
import pickle
with open('res_p16_unit.pickle', 'wb') as f:
    pickle.dump(res,f)

In [18]:

for idx,i in enumerate(res):
    if i[-1]is not None:  
        print(idx, i[-2]['uid'], i[3]['uid'],i[-1])


6 52164b70-6973-4844-af6a-76e8f1298d64 6c44a1a8-0785-43a0-90ab-7e21df2c57d9 [Error]could not convert string to float: ''
16 789efd09-04a2-45c7-84e5-47d3543695d1 35d602ae-9131-4291-a30c-49a40f32bbe4 [Error]invalid literal for int() with base 10: '$592'
21 77d8e381-01d0-4cf9-882e-e1162db2cff2 a983501d-2eec-486d-9661-e520c7c8af5e [Error]'(' was never closed (<string>, line 5)
26 cc4f089d-e7bf-4b00-a096-48a3eed39f86 29b63db7-4925-4629-b108-49bbb27fb565 [Error]list index out of range
28 22f634eb-a76a-424d-b8d3-3994dab52826 a81f1322-e74f-4e3c-a6cf-4b8d25d01cf5 [Error]invalid literal for int() with base 10: 'Selling, general and administrative'
34 32edf644-acb0-4260-9392-f0baa4253f5a bbe335ff-414f-48b3-8126-5a2c7c505de3 [Error]invalid literal for int() with base 10: 'Operating income'
53 8692f92d-668a-452d-be7a-2b491ee27585 7cba7400-af25-4f71-a49d-35d5eec31458 [Error]invalid literal for int() with base 10: '1,050'
54 8692f92d-668a-452d-be7a-2b491ee27585 94ef7822-a201-493e-b557-a640f4ea4d83 [E

In [36]:
item = devdf.iloc[0]
table = item['table']['table']
for q in item['questions']:        
    if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']: 
        print("==========")
        code = gen_code(llm, q, table)
        for l in table:
            print(l)
        print("\n", q['question'],"\n")
        print(code)
        r = exec_code(code, table)
        print(r, q['answer'], q['scale'])

['', '', 'Years Ended September 30,', '']
['', '2019', '2018', '2017']
['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9']
['Other', '44.1', '56.7', '70.8']
['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']

 What is the change in Other in 2019 from 2018? 

def run():
    table = [['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$ 1,452.4', '$ 1,146.2', '$ 1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]
    
    value_2019 = float(table[3][1])
    value_2018 = float(table[3][2])
    
    return (value_2019 - value_2018, 'million')
(-12.600000000000001, 'million') -12.6 million
['', '', 'Years Ended September 30,', '']
['', '2019', '2018', '2017']
['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9']
['Other', '44.1', '56.7', '70.8']
['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']

 What is the percentage change in Other in 2019 from 2018? 

def run():
    return (-22.22, 'percent

In [22]:
import re 
def remove_thousand_separators(text):
    # This regular expression matches numbers with commas as thousand separators
    return re.sub(r'(?<=\d),(?=\d{3})', '', text)

remove_thousand_separators('alma 123,432')

'alma 123432'

In [60]:
table

[['', 'December 31,', ''],
 ['', '2019', '2018'],
 ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'],
 ['Equity method investments (Note 1)', '9,254', '9,702'],
 ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'],
 ['Rent and other deposits', '6,106', '5,687'],
 ['Value added tax receivables, net, noncurrent', '592', '519'],
 ['Other', '6,723', '5,711'],
 ['', '$55,945', '$43,364']]

In [ ]:
print("==========")
code = gen_code(llm, q, table)
for l in table:
    print(l)
print("\n", q['question'],"\n")
print(code)
print(r, q['answer'], q['scale'])